In [52]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Twitter
import numpy as np
import pandas as pd
import pickle 
import os.path

'''
readCSV :CSV 파일을 읽는다. 고정포맷이 정해짐 
trainModel(ts =0.25): 입력된 모델에서 학습시킨다. ts 테스트 케이스 
predict(): 학습시킨 모델에 test데이터가 입력되면 예측한다. 
writePredictionResult(): 예측 결과를 파일로 출력한다. 
'''
twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

class FilterReviewSV:
    def __init__(self):
        self.model_lr_clf_name = 'model/model_lr_clf.sav'
        self.model_tfidf_vect_name = 'model/model_tfidf_vect.sav'
        
        self.isSetModel = False
        if not(self.isSetModel) and os.path.isfile(self.model_lr_clf_name) and os.path.isfile(self.model_tfidf_vect_name):
            print('모델을 불러오는중...')
            self.model_lr_clf = pickle.load(open(self.model_lr_clf_name, 'rb'))
            self.model_tfidf_vect = pickle.load(open(self.model_tfidf_vect_name, 'rb'))
            isSetModel = True
        else:
            print('모델이 없습니다..')
            

    def trainModel(self,filename,ts):
        data_df = pd.read_csv(filename,engine='python')
        X_train,X_test,y_train,y_test=train_test_split(data_df['sentence'],data_df['label'],test_size=0.25)
        self.model_tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
        self.model_tfidf_vect.fit(X_train)
        
        X_train_tfidf_vect = self.model_tfidf_vect.transform(X_train)
        X_test_tfidf_vect = self.model_tfidf_vect.transform(X_test)

        self.model_lr_clf = LogisticRegression(random_state=156)
        self.model_lr_clf.fit(X_train_tfidf_vect , y_train)
        pred = self.model_lr_clf.predict(X_test_tfidf_vect)
        print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

        pickle.dump(self.model_lr_clf, open(self.model_lr_clf_name, 'wb'))
        pickle.dump(self.model_tfidf_vect, open(self.model_tfidf_vect_name, 'wb'))
        isSetModel = True
    def readCSV(self,filename):
        self.data_df = pd.read_csv(filename,engine='python')
        self.test_df_tfidf_vect = self.model_tfidf_vect.transform(self.data_df['sentence'])
        
    def predict(self):
        self.y_pred = self.model_lr_clf.predict(self.test_df_tfidf_vect)
        self.df = pd.DataFrame({'pr_label':self.y_pred, 'sentence':self.data_df['sentence']})
        return self.y_pred
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='utf-8-sig',index=False)
        


In [53]:
fr = FilterReviewSV()
fr.trainModel(filename='MovieReview/MR_test2.csv',ts=0.25)

모델을 불러오는중...
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.787


In [54]:
fr = FilterReviewSV()
fr.readCSV(filename='MovieReview/MR_test.csv')
fr.predict()

fr.writePredictionResult('sample.csv')
#print(fr.y_pred)

모델을 불러오는중...


In [9]:
fr = FilterReviewSV()
fr.readCSV(tag=True,filename='MovieReview/MovieReview.csv', save=True)

모델을 불러오는중...
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다'
 '절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네' '마무리는 또 왜이래']
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다'
 '절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네' '마무리는 또 왜이래']


In [10]:
fr = FilterReviewSV()
fr.transDataLoad(tag=True)
fr.trainModel(ts=0.25)

모델을 불러오는중...
Number of right labeled points out of a total 49998 points : 0.6746 %


모델을 불러오는중...
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.' '원작의 긴장감을 제대로 살려내지못했다.'
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네'
 '액션이 없는데도 재미 있는 몇안되는 영화' '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?'
 '걍인피니트가짱이다.진짜짱이다♥' '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~']
['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.' '원작의 긴장감을 제대로 살려내지못했다.'
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네'
 '액션이 없는데도 재미 있는 몇안되는 영화' '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?'
 '걍인피니트가짱이다.진짜짱이다♥' '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~']
